In [40]:
%pip install scikit-learn scikit-image

Note: you may need to restart the kernel to use updated packages.


In [41]:
import os
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [42]:
# PREPPING DATA
input_dir = "/home/ajm76/OrcaSeis/data"
categories = ['noise', 'orca_call']

In [43]:
data = []
labels = []
for category_idx, category in enumerate(categories):
    cat_path = ""
    if category == 'noise':
        cat_path = 'noise_spectrograms' # actual code
#         cat_path = 'noise_test' # just for testing
    else:
        cat_path = 'orca_spectrograms' # actual code
#         cat_path = 'orca_test'
    for file in os.listdir(os.path.join(input_dir, cat_path)):
        img_path = os.path.join(input_dir, cat_path, file)
        img = imread(img_path)
        img = resize(img, (15, 15))
        data.append(img.flatten())
        labels.append(category_idx)

KeyboardInterrupt: 

In [31]:
data = np.asarray(data)
labels = np.asarray(labels)

In [33]:
# TRAIN/TEST SPLIT
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, 
                                                     shuffle=True, stratify=labels)

In [37]:
# TRAIN CLASSIFIER
classifier = SVC()

# actually training 12 classifiers, later choose best one
parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]

# # only for test datasets
# cv = StratifiedKFold(n_splits=3)

grid_search = GridSearchCV(classifier, parameters) # remove cv=cv parameter for actual 
grid_search.fit(x_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001]}])

In [39]:
# TEST PERFORMANCE
# choose best classifier out of the 12
best_estimator = grid_search.best_estimator_
y_prediction = best_estimator.predict(x_test) 
score = accuracy_score(y_prediction, y_test)

print('{}% of samples were correctly classified'.format(str(score * 100)))

100.0% of samples were correctly classified
